## Setting up clustering models

In [23]:
#Combine artcile files into one file
from os import listdir
allData = []
for filename in listdir('articles'):
    with open("articles/" + filename, "r") as read_file:
        allData.append(json.load(read_file))
        
with open("delfi_all.json", "w") as write_file:
    json.dump(allData, write_file)

In [1]:
import argparse
parser = argparse.ArgumentParser(description='Do document clustering experiment.')
parser.add_argument("--raw",      action='store_true')
parser.add_argument("--title",    action='store_true')
parser.add_argument("--intro",    action='store_true')
parser.add_argument("--stopword", action='store_true')
parser.add_argument("--stemer",   action='store_true')
parser.add_argument("--lemstop",  action='store_true')
parser.add_argument("--lemmer",   action='store_true')

parser.add_argument("--ngram",  type=int,   choices=range(1, 7),)
parser.add_argument("--min_df", type=int,   choices=range(1, 11),)
parser.add_argument("--max_df", type=float,) 
parser.add_argument("--dr",     type=int,   choices=range(1, 101),)

parser.add_argument('--methods', nargs='+', choices='km em ac aa aw db'.split(),
                    default='km em ac aa aw db'.split(),)
args = parser.parse_args('--lemmer'.split())
#args = parser.parse_args() 


import json
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from scipy.sparse import csr_matrix

def make_dataset(experiment_data, **kwargs):
    vectorizer = TfidfVectorizer(**kwargs)
    matrix = vectorizer.fit_transform(experiment_data)
    names = vectorizer.get_feature_names()
    print('>' + str(vectorizer))
    print(matrix.shape)
    return [{"matrix" : matrix, "names" : names}]  

data, all_tokens, morfs = ([],[],[])
with open("delfi_all.json", "r") as read_file:
    allData = json.load(read_file)
all_tokens = [" ".join(d["tokens"] + d["stop_tokens"]) for d in allData]
morfs = [re.findall("<word=\"(.*)\" lemma=\"(\w*).*\" type=\"(.*)\"", d["lemms"]) for d in allData]

datasets = []
#args = parser.parse_args('--'.split()) # For testing

datasets += make_dataset(all_tokens, analyzer = 'char_wb', ngram_range = (1,1))

# if args.raw:  
#     datasets += make_dataset(all_tokens)
# if args.title:
#     datasets += make_dataset([re.sub("[\W\d_]+", " ", d["title"]).lower() for d in data])
# if args.intro:
#     datasets += make_dataset([re.sub("[\W\d_]+", " ", d["intro"]).lower() for d in data])
# if args.stopword:
#     datasets += make_dataset([" ".join(d["tokens"])                    for d in data])
# if args.stemer:
#     datasets += make_dataset([" ".join(d["stems"]  + d["stop_stems"])  for d in data])

# if args.lemstop:
#     datasets += make_dataset([" ".join([l[0] for l in m if l[2]
#                    .startswith(("dkt", "vksm", "bdv", , "bendr", "būdn", "pad", "pusd", "tikr"))]) for m in morfs])
# if args.lemmer:
#     datasets += make_dataset([" ".join([l[1] for l in m]) for m in morfs])
    
# if args.ngram:
#     n = args.ngram
#     datasets += make_dataset(all_tokens, analyzer = 'char_wb', ngram_range = (n,n))
# if args.min_df:
#     df = args.min_df
#     datasets += make_dataset(all_tokens, min_df = df)
# if args.max_df:
#     df = args.max_df
#     datasets += make_dataset(all_tokens, max_df = df)
# if args.dr:
#     svd = TruncatedSVD(args.dr)
#     lsa = make_pipeline(svd, Normalizer(copy=False)) 
#     X = make_dataset(all_tokens)[0]
#     matrix = csr_matrix(lsa.fit_transform(X["matrix"]))
#     datasets += [{"matrix" : matrix, "names" : X["names"]}]

>TfidfVectorizer(analyzer='char_wb', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)
(4058, 93)


In [7]:
category_names = ['Auto', 'Veidai', 'Sportas', 'Mokslas', 'Verslas']
categorys  = np.array([category_names.index(d["categorys"]) for d in data])
len(data)

4058

In [4]:
def for_table (stats):
    np.unique(stats, return_counts=True)
    for i in np.unique(stats, return_counts=True):
        for j in i:
            print(j, end="\t")
        print()
#     for i in np.unique(stats, return_counts=True)[1]: # Do this in spreedsheet
#         print(i/len(stats)*100, end="\t")
    print()
    

        
#MINI TESTS
#
#TAGS
# dt = [len(d['tags']) for d in data]
# print('Tag stats:')
# print("Average amount of tags: {}".format(np.average(dt)))
# for i in np.unique(dt, return_counts=True):
#     for j in i:
#         print(j, end=" ")
#     print()
# print("".format(max))

#PARTS OF SPEECH
# morfs[0][1][2].split()[0]
# ps=[]
# for m in morfs:
#     for w in m:
#         word_type = re.split("\W+", w[2])[0]
# #         if word_type == "tikr":
# #             print(w)
#         ps.append(word_type)
# for_table(ps)
# len(ps)

#NGRAM MORE THEN 5
# for n in range(6,11):
#     print(n)
#     vectorizer = TfidfVectorizer(analyzer = 'char_wb', ngram_range = (n,n))
#     matrix = vectorizer.fit_transform(all_tokens)
#     print(matrix.shape)

#INPUT DATA STATS
# raw = all_tokens
# title = [re.sub("[\W\d_]+", " ", d["title"]).lower() for d in data]
# intro = [re.sub("[\W\d_]+", " ", d["intro"]).lower() for d in data]
# for i in [raw, title, intro]:
#     lengths = [len(j.split()) for j in i]
#     print(min(lengths), max(lengths), np.mean(lengths))

#TOP LEMMAS
# from sklearn.feature_extraction.text import CountVectorizer

# bin_vectorizer = CountVectorizer(binary = True)
# %time bX = bin_vectorizer.fit_transform([" ".join([l[1] for l in m]) for m in morfs])
# print(bX.shape)
# full_vectorizer = CountVectorizer()
# %time fX = full_vectorizer.fit_transform([" ".join([l[1] for l in m]) for m in morfs])
# print(fX.shape)

# # TODO max_df 0.9 - 0.1; min_df 2-10 (test if 1 make difference)

# n_docs = bX.shape[0]
# n_tokens = sum([len(d["tokens"] + d["stop_tokens"]) for d in data])
# terms = bin_vectorizer.get_feature_names()
# counts = bX.sum(axis=0).A1
# total_counts = fX.sum(axis=0).A1
# unique_counts = list(set(counts))
# biggest = sorted(unique_counts)[::-1]
# top = 0
# stats = []
# for i in biggest:
#     term_indices = np.where(counts == i)[0]

#     for j in term_indices:
#             top += 1
#             stats.append("{},{},{},{}%,{},{}%".format(top,
#                                         terms[j],
#                                         #j, # index in terms
#                                         total_counts[j],
#                                         total_counts[j]/n_tokens*100,
#                                         i, # count
#                                         i/n_docs*100)) #percent of all docs

# with open("lem_stats.csv", "w") as word_stats:
#     word_stats.write("\n".join(stats))

#STEM, WORD, LEM LENGTHS
#TODO test with stops, remove empty strings, test lems
# stems = [d["stems"] + d["stop_stems"] for d in data]
# flat_stems = [item for sublist in stems for item in sublist]
# stems_lens = [len(stem) for stem in flat_stems if len(stem) > 0]
# print(min(stems_lens), max(stems_lens), np.mean(stems_lens))
# word_lens = []
# for sub in all_tokens:
#     for item in sub.split():
#         word_lens.append(len(item))
#     #len(item) for sublist in all_tokens for item in sublist]
# print(min(word_lens), max(word_lens), np.mean(word_lens))
# lem_lens = []
# for sub in [" ".join([l[1] for l in m]) for m in morfs]:
#     for item in sub.split():
#         lem_lens.append(len(item))
# print(min(lem_lens), max(lem_lens), np.mean(lem_lens))

# 1-GRAM LETTERS

vectorizer = TfidfVectorizer(analyzer = 'char_wb', ngram_range = (1,1))
matrix = vectorizer.fit_transform(all_tokens)
print(vectorizer.get_feature_names())

[' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '²', '³', 'º', '¼', 'à', 'á', 'â', 'ä', 'å', 'ç', 'è', 'é', 'ë', 'ì', 'í', 'ï', 'ñ', 'ò', 'ó', 'ô', 'õ', 'ö', 'ø', 'ü', 'ā', 'ą', 'č', 'ė', 'ę', 'ī', 'į', 'ļ', 'ł', 'ņ', 'ō', 'œ', 'ś', 'š', 'ū', 'ų', 'ž', 'ʼ', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'х', 'ш', 'ы', 'э']


In [20]:
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
K = 5
jobs = -1

methods = []

if "km" in args.methods:
    KMtitle = "K-means"
    KMmodel = KMeans(n_clusters=K,
#                  max_iter=1,
#                  n_init=1,
                 n_jobs=jobs,
                 random_state=42,)
    methods += [{"model": KMmodel, "title": KMtitle}]
    
if "em" in args.methods:
    EMtitle = "Expectation–maximization"
    EMmodel = GaussianMixture(n_components=K,
                              covariance_type='diag',
#                         n_init=10,
                              random_state=42,)
    methods += [{"model": EMmodel, "title": EMtitle}]
    
if "ac" in args.methods:
    ACtitle = "Complete-linkage clustering"
    ACmodel = AgglomerativeClustering(n_clusters=K,
                                      linkage='complete',)
    methods += [{"model": ACmodel, "title": ACtitle}]
    
if "aa" in args.methods:
    AAtitle = "Average-linkage clustering"
    AAmodel = AgglomerativeClustering(n_clusters=K,
                                  linkage='average',)
    methods += [{"model": AAmodel, "title": AAtitle}]
    
if "aw" in args.methods:
    AWtitle = "Ward-linkage clustering"
    AWmodel = AgglomerativeClustering(n_clusters=K,
                                  linkage='ward',)
    methods += [{"model": AWmodel, "title": AWtitle}]
    
if "db" in args.methods:
    DBSCANtitle = "DBSCAN"
    DBSCANmodel = DBSCAN(n_jobs = jobs,)
    methods += [{"model": DBSCANmodel, "title": DBSCANtitle}]
print(methods)

[{'title': 'K-means', 'model': KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=-1, precompute_distances='auto',
    random_state=42, tol=0.0001, verbose=0)}, {'title': 'Expectation–maximization', 'model': GaussianMixture(covariance_type='diag', init_params='kmeans', max_iter=100,
        means_init=None, n_components=5, n_init=1, precisions_init=None,
        random_state=42, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)}, {'title': 'Complete-linkage clustering', 'model': AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
            connectivity=None, linkage='complete', memory=None,
            n_clusters=5, pooling_func='deprecated')}, {'title': 'Average-linkage clustering', 'model': AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
            connectivity=None, linkage='average', memory=None,
            n_clusters=5, pooli

## Set up evaluation functions

In [1]:
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import *
from scipy.stats import mode

def print_top_terms(model, terms):
#    print("Top terms per cluster:")
    centers = model.cluster_centers_ if isinstance(model, KMeans) else model.means_
    order_centroids = centers.argsort()[:, ::-1]
    for i in range(K):
        print("Cluster %d:" % i, end='')
        for ind in order_centroids[i, :10]:
            print(' %s' % terms[ind], end='')
        print()

def get_new_labels(clusters):
    new_labels = np.zeros_like(clusters)
    print("New labels:")
    for i in range(K):
        mask = (clusters == i)
        closest_category = mode(categorys[mask])[0][0]
        new_labels[mask] = closest_category
        print("{} -> {}({})".format(i, closest_category, category_names[closest_category]))
    print(np.bincount(new_labels))
    return new_labels

def print_metrics(y_pred):
    print("Rand        %.3f" %(adjusted_rand_score(categorys, y_pred)))
    print("Homogeneity %.3f" %(homogeneity_score(categorys, y_pred)))
    print("Completeness %.3f" %(completeness_score(categorys, y_pred)))

def plot_confusion_matrix(y_pred, title='clusters'):
    cm = confusion_matrix(categorys, y_pred)
    print(cm)

## Run models and metrics

In [22]:
def metrics_and_martix(clusters, m):
    print_metrics(clusters)
    plot_confusion_matrix(clusters, title=m['title'])
    new_labels = get_new_labels(clusters)
    print_metrics(new_labels)
    plot_confusion_matrix(new_labels, title=m['title'])
    
def analyse(m, data):
    model = m['model']
    print('\n' + m['title'] + " results")
    dataset = data["matrix"]
    if m['title'] == KMtitle:
        clusters = model.fit_predict(dataset)
        print(np.unique(clusters, return_counts=True)[1])
        
        print_top_terms(model, data["names"])
        metrics_and_martix(clusters, m)
        
    if m['title'] == EMtitle:
        model.fit(dataset.toarray())
        clusters = model.predict(dataset.toarray())
        print(np.unique(clusters, return_counts=True))
        
        print_top_terms(model, data["names"])
        metrics_and_martix(clusters, m)
        
    if m['title'] in [ACtitle, AAtitle, AWtitle]:
        clusters = model.fit_predict(dataset.toarray())
        print(np.unique(clusters, return_counts=True))
        
        metrics_and_martix(clusters, m)
        
    if m['title'] == DBSCANtitle:
        for e in [0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]:
            for m in [3, 4, 5, 6, 7, 8]:
                model.set_params(eps = e, min_samples = m,)
                clusters = model.fit_predict(dataset)
                
                results = np.unique(clusters, return_counts=True)
                if results[0][0] == -1: #if there was noise 
                    n_noise    = results[1][0]
                    n_clusters = np.sort(results[1][1:])[::-1]
                else:
                    n_noise    = 0      #if there was no noise 
                    n_clusters = np.sort(results[1])[::-1]
                print ("ε=%.1f min=%i: noise=%4i clusters=%3i top10=%s" 
                       %(e, m, n_noise, len(n_clusters), n_clusters[:10]))
    else:
        print(m)
        

In [23]:
import time

for dataset in datasets:
    print('>' + str(dataset["matrix"].shape))
    for method in methods:
        start_time = time.time()
        analyse(method, dataset)
        print (">", method['title'], time.time() - start_time, "to run")

>(4058, 18118)

K-means results
[ 459 1262  372  588 1377]
Cluster 0:  spo spor mpio čemp pion empi  čem port varž nkty
Cluster 1: inti gas  film dain sti   dai nti   žmo ilma  fil
Cluster 2: gtyn ngty  run rung vart tynė ynės nės  įvar ungt
Cluster 3: omob utom mobi obil tomo ilis bili auto  aut vair
Cluster 4: cija  pro toja ojas darb eura monė  eur otoj  dar
Rand        0.415
Homogeneity 0.512
Completeness 0.558
[[122  32   0 562 179]
 [  3 740   0   1  35]
 [331  38 372   1  18]
 [  2 420   0   4 411]
 [  1  32   0  20 734]]
New labels:
0 -> 2(Sportas)
1 -> 1(Veidai)
2 -> 2(Sportas)
3 -> 0(Auto)
4 -> 4(Verslas)
[ 588 1262  831    0 1377]
Rand        0.460
Homogeneity 0.499
Completeness 0.601
[[562  32 122   0 179]
 [  1 740   3   0  35]
 [  1  38 703   0  18]
 [  4 420   2   0 411]
 [ 20  32   1   0 734]]
{'title': 'K-means', 'model': KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=-1, precompute_distances='auto',
    random